In [1]:
import os
import pyurdme
import pyurdme.nsmsolver
import dolfin
import numpy
import scipy.io as spio
import matplotlib.pyplot as plt
import math
from scipy import integrate
import datetime as dt

In [2]:
class Example2(pyurdme.URDMEModel):
    """ The reversible reaction A+B <->C in 3D.  """
    
    def __init__(self,voxel_size=0.1e-6, val=1e-20, sigma=1e-9):
        
        pyurdme.URDMEModel.__init__(self,name="Example2")

        gamma = 1e-12

        # Substrate and enzymes
        S1  = pyurdme.Species(name="S1",reaction_radius=sigma,diffusion_constant=gamma)
        S11 = pyurdme.Species(name="S11",reaction_radius=sigma,diffusion_constant=gamma)
        S12 = pyurdme.Species(name="S12",reaction_radius=sigma,diffusion_constant=gamma)
        S2  = pyurdme.Species(name="S2",reaction_radius=sigma,diffusion_constant=gamma)
        self.add_species([S1,S11,S12,S2])

        pi = math.pi
        self.voxel_size = voxel_size
        L = 1e-6
        h = voxel_size
        nx = int(L/h)

        N = nx*nx*nx
        self.mesh = pyurdme.URDMEMesh.generate_cube_mesh(L,nx,nx,nx)
       
        # Microscopic association and disassociation rate
        kr  = pyurdme.Parameter(name="kr",expression=val)
        kd  = pyurdme.Parameter(name="kd",expression=10.0)

        self.add_parameter([kr,kd])
    
        # Reactions
        R1 = pyurdme.Reaction(name="R1",reactants={S1:1},products={S11:1,S12:1}, massaction=True, rate=kd)
        R2 = pyurdme.Reaction(name="R2",reactants={S11:1,S12:1},products={S2:1}, massaction=True, rate=kr)
        self.add_reaction([R1,R2])
        
        # Scatter the molecules over the mesh
        self.set_initial_condition_scatter({S1:100})

        # Time span of the simulation
        self.timespan(numpy.arange(0,2.0,0.05))
 

In [3]:
from pyurdme.rdsimsolver import MMMSSolver 
model = Example2(voxel_size=0.3e-6)
solver = MMMSSolver(model, min_micro_timestep=1e-4)
#solver.serialize("urdmemodel.mat")
#solver.create_input_file("Example.json")
#solver._write_mesh_file("urdmemesh.h5")
res = solver.run(number_of_trajectories=100)

In [4]:
!ls 

Example.json   Example2.py	      out.h5	     urdmemesh.h5
Example2.json  Hybrid_examples.ipynb  testoutput.h5  urdmemodel.mat


In [5]:
! rdsim Example.json urdmemodel.mat urdmemesh.h5 output.h5 

In [6]:
!ls -l output.h5

-rw-r--r-- 1 fenics fenics 234808 Mar  9 18:33 output.h5


In [7]:
from pyurdme.rdsimsolver import RDSIMResult
res = RDSIMResult(model, "output.h5")

In [4]:
res.get_summary_statistic("S1")

array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
       100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
       100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100])

In [5]:
particles = res.get_particles("S1",0)

In [6]:
print particles["unique_ids"]

[[100]
 [ 99]
 [ 98]
 [ 97]
 [ 46]
 [ 45]
 [ 44]
 [ 43]
 [ 42]
 [ 41]
 [ 40]
 [ 39]
 [ 38]
 [ 37]
 [ 36]
 [ 35]
 [ 34]
 [ 33]
 [ 32]
 [ 31]
 [ 30]
 [ 29]
 [ 28]
 [ 27]
 [ 26]
 [ 25]
 [ 24]
 [ 23]
 [ 10]
 [  9]
 [  8]
 [  7]
 [  6]
 [  5]
 [  1]
 [  2]
 [  3]
 [  4]
 [ 11]
 [ 12]
 [ 13]
 [ 14]
 [ 15]
 [ 16]
 [ 17]
 [ 18]
 [ 19]
 [ 20]
 [ 21]
 [ 22]
 [ 47]
 [ 48]
 [ 49]
 [ 50]
 [ 51]
 [ 52]
 [ 53]
 [ 54]
 [ 55]
 [ 56]
 [ 57]
 [ 58]
 [ 59]
 [ 60]
 [ 61]
 [ 62]
 [ 63]
 [ 64]
 [ 65]
 [ 66]
 [ 67]
 [ 68]
 [ 69]
 [ 70]
 [ 71]
 [ 72]
 [ 73]
 [ 74]
 [ 75]
 [ 76]
 [ 77]
 [ 78]
 [ 79]
 [ 80]
 [ 81]
 [ 82]
 [ 83]
 [ 84]
 [ 85]
 [ 86]
 [ 87]
 [ 88]
 [ 89]
 [ 90]
 [ 91]
 [ 92]
 [ 93]
 [ 94]
 [ 95]
 [ 96]]


In [11]:
resfile = res.get_species("S1",0)

<HDF5 file "output.h5" (mode r)>


In [12]:
r= resfile.get("Trajectories/0/Type_S1/positions_1")

In [13]:
print numpy.array(r)

[[  9.72610757e-07   8.39990531e-07   8.52619923e-07]
 [  4.94302015e-07   8.05695714e-07   9.18535205e-07]
 [  4.72263490e-07   9.02694386e-07   9.92035111e-07]
 [  6.64715017e-07   8.42734938e-07   8.92265838e-07]
 [  1.53314674e-07   3.94284067e-08   7.05039562e-07]
 [  9.56960575e-07   8.16421128e-07   2.63452997e-07]
 [  9.91350624e-07   9.99890565e-07   3.48050731e-07]
 [  2.75220715e-07   9.23654623e-07   2.96606758e-07]
 [  1.72367689e-07   7.65729290e-07   2.69282132e-07]
 [  2.39060565e-08   8.98930780e-07   2.21612385e-07]
 [  8.29415063e-07   4.59793636e-07   2.77621580e-07]
 [  9.45483741e-07   4.83039451e-07   2.67529350e-07]
 [  2.42454052e-07   5.76540808e-07   1.73924319e-07]
 [  1.77761659e-08   5.14981896e-07   2.25319242e-07]
 [  3.07882973e-08   5.25036744e-07   2.69835484e-07]
 [  1.18116864e-07   1.97267349e-07   2.80235251e-07]
 [  3.34095369e-07   2.63730446e-07   3.48597710e-07]
 [  1.76196091e-08   3.99294299e-07   5.05663784e-07]
 [  7.96634368e-08   3.15136

In [7]:
res.display_particles(["S1"],1)